In [ ]:
# import 'Pandas' 
import pandas as pd 

# import 'Numpy' 
import numpy as np

# import subpackage of Matplotlib
import matplotlib.pyplot as plt

# import 'Seaborn' 
import seaborn as sns

# to suppress warnings 
from warnings import filterwarnings
filterwarnings('ignore')

# display all columns of the dataframe
pd.options.display.max_columns = None

# display all rows of the dataframe
pd.options.display.max_rows = None
 
# to display the float values upto 6 decimal places     
pd.options.display.float_format = '{:.6f}'.format

# import train-test split 
from sklearn.model_selection import train_test_split

# import various functions from statsmodels
import statsmodels
import statsmodels.api as sm
import statsmodels.stats.api as sms
from statsmodels.graphics.gofplots import qqplot

# import 'stats'
from scipy import stats

# 'metrics' from sklearn is used for evaluating the model performance
from sklearn.metrics import mean_squared_error

# import functions to perform feature selection
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.feature_selection import RFE

# import function to perform linear regression
from sklearn.linear_model import LinearRegression

# import functions to perform cross validation
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [ ]:
# import train-test,cross validation,gridsearch cv
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

# importing classification models
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.pipeline import Pipeline

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path='/content/drive/MyDrive/df_final1/train_rf.csv'

train=pd.read_csv(path)
train.drop('Unnamed: 0',axis=1,inplace=True)
train.head()

,loan_amnt,int_rate,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,default_ind,term_ 60 months,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,initial_list_status_w,grade,sub_grade,emp_length
0,17325,12.120000,40000.000000,31.410000,0,0,10,0,32236,77.700000,14,0.000000,100408.000000,41500.000000,0,0,1,0,0,0,0,0,1,1,1,7,10.000000
1,35000,23.990000,160000.000000,12.480000,0,2,15,1,14600,42.200000,36,0.000000,78073.000000,34600.000000,0,0,1,0,0,0,0,1,0,0,5,26,2.000000
2,10000,12.120000,60000.000000,7.580000,2,0,8,0,39414,49.800000,32,0.000000,82421.000000,23900.000000,0,0,0,0,0,0,1,0,1,0,1,7,5.000000
3,10000,13.220000,35000.000000,8.090000,0,0,7,0,8907,61.900000,7,0.000000,82421.000000,23900.000000,0,0,0,0,0,0,1,0,0,0,2,11,3.000000
4,35000,23.400000,100000.000000,31.690000,2,0,10,0,31528,82.700000,31,0.000000,74319.000000,61200.000000,0,1,0,0,0,0,1,0,1,0,4,24,10.000000


# Data Info

In [ ]:
train.isnull().sum().sum()

0

In [ ]:
X_train=train.drop('default_ind',axis=1)

y_train=train['default_ind']

In [ ]:
X_train.shape,y_train.shape

((558814, 26), (558814,))

In [ ]:
X_train.columns,len(X_train.columns)

(Index(['loan_amnt', 'int_rate', 'annual_inc', 'dti', 'delinq_2yrs',
        'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
        'total_acc', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim',
        'term_ 60 months', 'home_ownership_MORTGAGE', 'home_ownership_NONE',
        'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT',
        'verification_status_Source Verified', 'verification_status_Verified',
        'initial_list_status_w', 'grade', 'sub_grade', 'emp_length'],
       dtype='object'), 26)

# **RECURSIVE FEATURE ELIMINATION**

In [ ]:
# get a list of models to evaluate
def get_models():
    models = dict()
    for i in range(2, 27):
        rfe = RFE(estimator=RandomForestClassifier(criterion='entropy',random_state = 10,class_weight='balanced',max_depth=8), n_features_to_select=i)
        model = RandomForestClassifier(criterion='entropy',random_state = 10,class_weight='balanced',max_depth=8)
        models[str(i)] = Pipeline(steps=[('s',rfe),('m',model)])
    return models
 
# evaluate a give model using cross-validation
def evaluate_model(model,X,y):
    cv = KFold(n_splits=3,shuffle=True,random_state=10)
    scores = cross_val_score(model,X,y,scoring='recall', cv=cv, n_jobs=-1, error_score='raise')
    return scores
 

# get the models to evaluate
models = get_models()

# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, X_train, y_train)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, np.mean(scores), np.std(scores,ddof=1)))

# plot model performance for comparison
plt.boxplot(results, labels=names, showmeans=True)
plt.show()

>2 0.797 (0.023)
>3 0.793 (0.010)
>4 0.770 (0.007)
>5 0.753 (0.020)
>6 0.754 (0.005)
>7 0.749 (0.010)
>8 0.745 (0.008)
>9 0.749 (0.008)
>10 0.742 (0.003)
>11 0.741 (0.008)
>12 0.740 (0.003)
>13 0.741 (0.010)
>14 0.740 (0.004)
>15 0.735 (0.011)
>16 0.743 (0.007)
>17 0.740 (0.008)


In [ ]:
rfe = RFE(estimator=DecisionTreeClassifier(random_state = 10,class_weight='balanced'), n_features_to_select=9)
# fit RFE
rfe.fit(X_train, y_train)
# summarize all features
for i in range(X_train.shape[1]):
	print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))


Column: 0, Selected True, Rank: 1.000
Column: 1, Selected True, Rank: 1.000
Column: 2, Selected True, Rank: 1.000
Column: 3, Selected True, Rank: 1.000
Column: 4, Selected False, Rank: 8.000
Column: 5, Selected False, Rank: 6.000
Column: 6, Selected False, Rank: 3.000
Column: 7, Selected False, Rank: 9.000
Column: 8, Selected True, Rank: 1.000
Column: 9, Selected True, Rank: 1.000
Column: 10, Selected True, Rank: 1.000
Column: 11, Selected False, Rank: 7.000
Column: 12, Selected True, Rank: 1.000
Column: 13, Selected True, Rank: 1.000
Column: 14, Selected False, Rank: 15.000
Column: 15, Selected False, Rank: 12.000
Column: 16, Selected False, Rank: 18.000
Column: 17, Selected False, Rank: 17.000
Column: 18, Selected False, Rank: 13.000
Column: 19, Selected False, Rank: 16.000
Column: 20, Selected False, Rank: 10.000
Column: 21, Selected False, Rank: 11.000
Column: 22, Selected False, Rank: 4.000
Column: 23, Selected False, Rank: 14.000
Column: 24, Selected False, Rank: 2.000
Column: 25

In [ ]:

rfe_model = RFE(estimator=DecisionTreeClassifier(random_state = 10,class_weight='balanced'), n_features_to_select =9)

# fit the RFE model on the training dataset using fit()
rfe_model = rfe_model.fit(X_train, y_train)

# create a series containing feature and its corresponding rank obtained from RFE
# 'ranking_' returns the rank of each variable after applying RFE
# pass the ranks as the 'data' of a series
# 'index' assigns feature names as index of a series 
feat_index = pd.Series(data = rfe_model.ranking_, index = X_train.columns)

# select the features with rank = 1
# 'index' returns the indices of a series (i.e. features with rank=1) 
signi_feat_rfe = feat_index[feat_index==1].index

# print the significant features obtained from RFE
print(signi_feat_rfe)

Index(['loan_amnt', 'int_rate', 'annual_inc', 'dti', 'revol_bal', 'revol_util',
       'total_acc', 'tot_cur_bal', 'total_rev_hi_lim'],
      dtype='object')


In [ ]:
X_train.columns

Index(['loan_amnt', 'int_rate', 'annual_inc', 'dti', 'delinq_2yrs',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim',
       'term_ 60 months', 'home_ownership_MORTGAGE', 'home_ownership_NONE',
       'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT',
       'verification_status_Source Verified', 'verification_status_Verified',
       'initial_list_status_w', 'grade', 'sub_grade', 'emp_length'],
      dtype='object')

# **SEQUENTIAL FEATURE SELECTION**

## **SFS- Forward (Decision Tree)**

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
# ada = AdaBoostClassifier()
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
from sklearn import tree
dt= tree.DecisionTreeClassifier()
    
dt_forward = sfs(estimator=dt, k_features = 'best', forward=True,
                     verbose=2, scoring='f1_weighted')

sfs_forward = dt_forward.fit(X_train_rf, y_train_rf)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   13.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  1.6min finished

[2021-09-16 15:07:06] Features: 1/27 -- score: 0.9193025491040938[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  26 out of  26 | elapsed:  1.7min finished

[2021-09-16 15:08:50] Features: 2/27 -- score: 0.9196950037692095[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:  2.3min finished

[2021-09-16 15:11:10] Features: 3/27 -- score: 0.9198062716657243[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  

In [ ]:
# print the selected feature names when k_features = 'best'
print('Features selelected using forward selection are: ')
print(dt_forward.k_feature_names_)

# print the R-squared value
print('\nScore: ', dt_forward.k_score_)

Features selelected using forward selection are: 
('int_rate', 'open_acc', 'verification_status_Verified', 'sub_grade')

Score:  0.9199273194762208


## **SFS- Backward (Decision Tree)**

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
# ada = AdaBoostClassifier()
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
from sklearn import tree
dt= tree.DecisionTreeClassifier()
    
rf_backward = sfs(estimator=dt, k_features = 10, forward= True,
                     verbose=3, scoring='f1_weighted')

sfs_backward = rf_backward.fit(X_train_rf, y_train_rf)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   16.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  1.5min finished

[2021-09-20 15:13:49] Features: 1/10 -- score: 0.9193025491040938[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   14.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  26 out of  26 | elapsed:  1.6min finished

[2021-09-20 15:15:23] Features: 2/10 -- score: 0.9196950004535104[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   17.8s remaining:    0.0s
[Para

In [ ]:
# print the selected feature names when k_features = 'best'
print('Features selelected using forward selection are: ')
print(rf_backward.k_feature_names_)

# print the R-squared value
print('\nScore: ', rf_backward.k_score_)

Features selelected using forward selection are: 
('int_rate', 'open_acc', 'home_ownership_NONE', 'home_ownership_OTHER', 'home_ownership_OWN', 'verification_status_Source Verified', 'verification_status_Verified', 'initial_list_status_w', 'grade', 'sub_grade')

Score:  0.9193673417505851


## **SFS- Forward (Random Forest)**

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
# ada = AdaBoostClassifier()
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
from sklearn import tree
dt= tree.DecisionTreeClassifier()
    
rf_forward = sfs(estimator=rf, k_features = 10, forward= True,
                     verbose=3, scoring='f1_weighted')

sfs_forward = rf_forward.fit(X_train_rf, y_train_rf)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 13.9min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 18.7min remaining:    0.0s
[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 102.4min finished

[2021-09-21 09:12:25] Features: 1/10 -- score: 0.9193155609060482[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  7.0min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 11.4min remaining:    0.0s
[Parallel(n_jobs=1)]: Done  26 out of  26 | elapsed: 96.4min finished

[2021-09-21 10:48:50] Features: 2/10 -- score: 0.9197113264484653[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  5.7min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 10.6min remaining:    0.0s
[Par

In [ ]:
# print the selected feature names when k_features = 'best'
print('Features selelected using forward selection are: ')
print(rf_forward.k_feature_names_)

# print the R-squared value
print('\nScore: ', rf_forward.k_score_)